// TO DO: rewind the idea

Using pre-trained SSD implementation: Model ZOO

In [ ]:
! git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 74962, done.
remote: Counting objects: 100% (450/450), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 74962 (delta 266), reused 393 (delta 235), pack-reused 74512
Receiving objects: 100% (74962/74962), 580.53 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (53166/53166), done.


In [ ]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cd models/research && \
    cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .

In [ ]:
import os
import imageio
import pathlib
import tensorflow as tf

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
!wget -nc https://lazyprogrammer.me/cnn_class2_videos.zip

--2022-07-07 18:22:18--  https://lazyprogrammer.me/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  1.24MB/s    in 1.6s    

2022-07-07 18:22:20 (1.24 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [ ]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             


In [ ]:
! ls

models	sample_data


In [ ]:
INPUT_VIDEOS = ['catdog', 'safari', 'traffic']

In [ ]:
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin=url,
    untar=True)

386527459/386527459 [==============================] - 5s 0us/step


In [ ]:
PATH_TO_MODEL_DIR

'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

In [ ]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url,
    untar=False)

5056/5056 [==============================] - 0s 0us/step


In [ ]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [ ]:
! head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


In [ ]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print(' took {} seconds'.format(elapsed_time))

Loading model... took 27.891950607299805 seconds


In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS,
    use_display_name=True)

category_index

In [ ]:
def detect_objects_in_image(image_np):

  # make proper size for neural network
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  # actual detection
  detections = detect_fn(input_tensor)

  # reformat back the additional dimension
  num_detections = int(detections.pop('num_detections'))
  detections = {
                  key: value[0, :num_detections].numpy()
                  for key, value in detections.items()
                }

  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  # make copy for annotations
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
  
  return image_np_with_detections

In [ ]:
def detect_objects_in_video(input_video):
  print(f"Running inference video for {input_video}.mp4", end='')

  video_reader = imageio.get_reader(f'{input_video}.mp4')
  video_writer = imageio.get_writer(f'{input_video}_annotated.mp4', fps=10)

  # process each frame in the video
  t0 = time.time()
  n_frames = 0
  for frame in video_reader:
    n_frames += 1
    new_frame = detect_objects_in_image(frame)  # API function

    video_writer.append_data(new_frame)

  fps = n_frames / (time.time() - t0)
  print("Frames processed: %s, Speed: %s fps" % (n_frames, fps))

  video_writer.close()

In [ ]:
detect_objects_in_video(INPUT_VIDEOS[0])